**Train Model**

In [4]:
!pip install -Uqq fastai
!pip install -Uqq duckduckgo_search

from duckduckgo_search import ddg_images
from fastcore.all import *

def search_images(term, max_images=200): return L(ddg_images(term, max_results=max_images)).itemgot('image')

from fastdownload import download_url
from fastai.vision.all import *

searches = 'Elon Musk', 'Yilong Ma'
path = Path('Elon Musk')
from time import sleep

for o in searches:
    dest = (path/o)
    dest.mkdir(exist_ok=True, parents=True)
    download_images(dest, urls=search_images(f'{o} Elon Musk'))
    download_images(dest, urls=search_images(f'{o} Yilong Ma'))
    resize_images(path/o, max_size=20, dest=path/o)
    
failed = verify_images(get_image_files(path))
failed.map(Path.unlink)
len(failed)

dls = DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files, 
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=[Resize(192, method='squish')]
).dataloaders(path)

!pip install -Uqq fastbook
from fastbook import *
from fastai.vision.widgets import *

learn = vision_learner(dls, resnet34, metrics=error_rate)

learn.export()

/opt/conda/lib/python3.7/site-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and may be removed in the future, "
/opt/conda/lib/python3.7/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth


  0%|          | 0.00/83.3M [00:00<?, ?B/s]

In [5]:
path = Path()
path.ls(file_exts='.pkl')

(#1) [Path('export.pkl')]

**Export Model**

In [6]:
!pip install -Uqq fastai
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

from fastbook import *
from fastai.vision.widgets import *

learn_inf = load_learner('export.pkl')

categories = ('1','2','3','4','5','7','8')

def classify_image(img) :
    pred, idx, probs = learn_inf.predict(img)
    return dict(zip(categories, map(float, probs)))

!pip install -Uqq gradio
import gradio as gr

In [9]:
btn_upload = widgets.FileUpload()
btn_upload

from fastdownload import download_url
download_url('https://upload.wikimedia.org/wikipedia/commons/thumb/3/34/Elon_Musk_Royal_Society_%28crop2%29.jpg/1200px-Elon_Musk_Royal_Society_%28crop2%29.jpg','Elon.jpg', show_progress=False)

from fastai.vision.all import *
im = Image.open('Elon.jpg')
im.to_thumb(256,256)

btn_upload = SimpleNamespace(data = ['Elon.jpg'])
img = PILImage.create(btn_upload.data[-1])

out_pl = widgets.Output()
out_pl.clear_output()
with out_pl: display(img.to_thumb(128,128))

pred,pred_idx,probs = learn_inf.predict(img)

lbl_pred = widgets.Label()
lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'
lbl_pred

btn_run = widgets.Button(description='Classify')
btn_run

def on_click_classify(change):
    img = PILImage.create(btn_upload.data[-1])
    out_pl.clear_output()
    with out_pl: display(img.to_thumb(128,128))
    pred,pred_idx,probs = learn_inf.predict(img)
    lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'

btn_run.on_click(on_click_classify)

btn_upload = widgets.FileUpload()

VBox([widgets.Label('Input picture of Elon Musk or Yilong Ma!'), 
      btn_upload, btn_run, out_pl, lbl_pred])